In [1]:
from pyspark.sql.types import *

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [21]:
header_schema = StructType([
  StructField("DocEntry", IntegerType(), False),
  StructField("DocDate", StringType(), False),
  StructField("CardCode", StringType(), False),
  StructField("CardName", StringType(), False),
  StructField("DocTotal", DoubleType(), False),
])
detail_schema = StructType([
  StructField("DocEntry", IntegerType(), False),
  StructField("ItemCode", StringType(), False),
  StructField("Dscription", StringType(), False),
  StructField("Price", DoubleType(), False),
  StructField("Quantity", IntegerType(), False),
  StructField("LineTotal", DoubleType(), False),
])
item_schema = StructType([
  StructField("ItemCode", StringType(), False),
  StructField("ItemName", StringType(), False),
  StructField("FrgnName", StringType(), False),
  StructField("ItmsGrpCod", StringType(), False),
])

headers = spark.read.option("delimiter", "|").csv('./data/2018-01/sohd*.csv', schema=header_schema, header=True).drop("CardName")
details = spark.read.option("delimiter", "|").csv('./data/2018-01/sodt*.csv', schema=detail_schema, header=True)
items_master = spark.read.option("delimiter", "|").csv('./data/items-master.csv', schema=item_schema, header=True).select("ItemCode", "ItmsGrpCod")
# headers.show()
# details.show()
# items_master.show()
joined_df = headers.join(details, "DocEntry").join(items_master, "ItemCode")
joined_df.show(20)

+--------+--------+--------------------+--------+--------+--------------------+-------+--------+---------+----------+
|ItemCode|DocEntry|             DocDate|CardCode|DocTotal|          Dscription|  Price|Quantity|LineTotal|ItmsGrpCod|
+--------+--------+--------------------+--------+--------+--------------------+-------+--------+---------+----------+
| 5402648|     148|2018-01-01T00:43:...| C550025| 1391.58|กระเช้า ราคา 1800...|1258.92|       1|  1258.92|       112|
| 3702298|     148|2018-01-01T00:43:...| C550025| 1391.58|  อีซี่บาร์ รสไข่แดง|  47.38|       1|    47.38|       111|
| 1109076|     148|2018-01-01T00:43:...| C550025| 1391.58|ยำสาหร่ายSHIMANTO...|  54.69|       1|    54.69|       108|
| 4104087|     148|2018-01-01T00:43:...| C550025| 1391.58|น้ำส้มกรีนสปอต 26...|   9.37|       1|     9.37|       110|
| 2101016|     148|2018-01-01T00:43:...| C550025| 1391.58|         บัวลอยเผือก|  21.22|       1|    21.22|       109|
| 0101690|     463|2018-01-01T02:06:...| E000676|  1289.

+-------+------------------+------------------+--------------------+--------+--------------------+------------------+--------------------+-----------------+--------+-----------------+--------------------+--------+------------------+
|summary|          ItemCode|          DocEntry|             DocDate|CardCode|            CardName|          DocTotal|          Dscription|            Price|Quantity|        LineTotal|            ItemName|FrgnName|        ItmsGrpCod|
+-------+------------------+------------------+--------------------+--------+--------------------+------------------+--------------------+-----------------+--------+-----------------+--------------------+--------+------------------+
|  count|           6876853|           6876853|             6876853| 6876853|             6876853|           6876853|             6876853|          6876853| 6876853|          6876853|             6876853|       0|           6876853|
|   mean|1948772.2718621439| 649927.6835982971|                null|